# FreqAI GPU Backtest - Remote Setup

**Enable GPU:** Runtime > Change runtime type > GPU (T4)

**Then:** Run all cells (Runtime > Run all)

In [ ]:
# 1. Check GPU
!nvidia-smi --query-gpu=name,memory.total --format=csv || echo '❌ GPU not found. Enable GPU!'

In [ ]:
%%capture
# 2. Install SSH
!apt-get update -qq
!apt-get install -y openssh-server

In [ ]:
# 3. Setup SSH
import secrets, string
password = ''.join(secrets.choice(string.ascii_letters + string.digits) for _ in range(16))
!mkdir -p /var/run/sshd
!echo "root:{password}" | chpasswd
!sed -i 's/#PermitRootLogin prohibit-password/PermitRootLogin yes/' /etc/ssh/sshd_config
!sed -i 's/#PasswordAuthentication yes/PasswordAuthentication yes/' /etc/ssh/sshd_config
!service ssh restart
print(f'✅ SSH ready | Password: {password}')

In [ ]:
# 4. Download ngrok
import os
os.chdir('/content')
!wget -q https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xzf ngrok-v3-stable-linux-amd64.tgz
!chmod +x ngrok
!ls -lh ngrok
!file ngrok
print('✅ ngrok downloaded')
print('\n📋 Get your token: https://dashboard.ngrok.com/get-started/your-authtoken')

In [ ]:
# 5. Configure ngrok
import getpass, os
ngrok_token = getpass.getpass('🔑 Enter ngrok token: ')
config_dir = '/root/.config/ngrok'
!mkdir -p {config_dir}
config = f'version: "2"\nauthtoken: {ngrok_token}\n'
with open(f'{config_dir}/ngrok.yml', 'w') as f:
    f.write(config)
print(f'✅ Config saved to {config_dir}/ngrok.yml')
!cat {config_dir}/ngrok.yml

In [ ]:
# 6. Test ngrok manually
import os
os.chdir('/content')
print('🧪 Testing ngrok...')
!./ngrok version || echo '❌ ngrok test failed'
print('\n🧪 Testing with config...')
!timeout 3 ./ngrok tcp 22 > /tmp/ngrok.log 2>&1 || echo 'Timeout OK'
print('\n📄 ngrok output:')
!cat /tmp/ngrok.log

In [ ]:
# 7. Start tunnel (with error capture)
import subprocess, time, requests, os
os.chdir('/content')
print('⏳ Starting ngrok tunnel...')
print('Working dir:', os.getcwd())
print('Executable:', '/content/ngrok')
print()

# Start ngrok with output capture
with open('/tmp/ngrok_startup.log', 'w') as log:
    proc = subprocess.Popen(
        ['/content/ngrok', 'tcp', '22'],
        stdout=log,
        stderr=subprocess.STDOUT,
        cwd='/content'
    )

# Wait for API to be ready
time.sleep(8)

# Check if process is still running
if proc.poll() is not None:
    print('❌ ngrok exited! Return code:', proc.returncode)
    print('\n📄 Startup log:')
    !cat /tmp/ngrok_startup.log
    raise RuntimeError('ngrok failed to start')

print('✅ ngrok process running (PID:', proc.pid, ')')
print('⏳ Waiting for API...')

# Try to get tunnel URL
tunnel_url = None
for i in range(25):
    try:
        time.sleep(1)
        r = requests.get('http://127.0.0.1:4040/api/tunnels', timeout=2)
        if r.status_code == 200:
            tunnels = r.json().get('tunnels', [])
            if tunnels:
                tunnel_url = tunnels[0]['public_url']
                break
    except Exception as e:
        if i % 5 == 0:
            print(f'  [{i}/25] Waiting... ({type(e).__name__})')

if tunnel_url:
    print('\n' + '='*70)
    print('✅ TUNNEL ACTIVE')
    print('='*70)
    print(f'URL: {tunnel_url}')
    print(f'Password: {password}')
    print('='*70)
    print('\nRun on LOCAL machine:\n')
    print(f'python tools/backtest_executor.py \\')
    print(f'  --tunnel-url "{tunnel_url}" \\')
    print(f'  --password "{password}" \\')
    print(f'  --strategy FreqAIHybridStrategy \\')
    print(f'  --timerange 20250901-20251012 \\')
    print(f'  --pairs BTC/USDT:USDT')
    print('='*70)
else:
    print('\n❌ Could not get tunnel URL')
    print('\n📄 Check ngrok web UI manually:')
    print('!curl http://127.0.0.1:4040/api/tunnels')
    print('\n📄 Check ngrok log:')
    !cat /tmp/ngrok_startup.log

In [ ]:
%%capture
# 8. Clone repo
import os
os.chdir('/content')
!rm -rf freqai-futures-strategy
!git clone -q https://github.com/aminak58/freqai-futures-strategy.git
os.chdir('/content/freqai-futures-strategy')
!pip install -q ta-lib-binary 'freqtrade[freqai]'

In [ ]:
# 9. Verify
import os
print(f'✅ Ready: {os.getcwd()}')
!ls -lh | head -10

In [ ]:
# 10. Optional: Mount Drive
from google.colab import drive
import zipfile, os
drive.mount('/content/drive')
zip_path = '/content/drive/MyDrive/FreqAI/binance_data.zip'
if os.path.exists(zip_path):
    print('📦 Extracting data...')
    os.chdir('/content/freqai-futures-strategy')
    os.makedirs('user_data/data', exist_ok=True)
    with zipfile.ZipFile(zip_path) as z:
        z.extractall('user_data/data/')
    print('✅ Data ready')
else:
    print('ℹ️  No data in Drive')

In [ ]:
# 11. Keep alive
import time
from IPython.display import clear_output
print('🔄 Session active - Tunnel running')
print('='*70)
print(f'URL: {tunnel_url}')
print(f'Password: {password}')
print('='*70)
print('Updates every 5min\n')
c = 0
while True:
    try:
        time.sleep(300)
        c += 1
        clear_output(wait=True)
        print(f'🔄 Active #{c} ({c*5}min)')
        print('='*70)
        print(f'URL: {tunnel_url}')
        print(f'Password: {password}')
        print('='*70)
        print(time.strftime('%H:%M:%S'))
    except KeyboardInterrupt:
        break

---
## Troubleshooting

In [ ]:
# Check tunnel status
!curl -s http://127.0.0.1:4040/api/tunnels | python3 -m json.tool

In [ ]:
# Check ngrok process
!ps aux | grep ngrok

In [ ]:
# Check ngrok logs
!cat /tmp/ngrok_startup.log 2>/dev/null || echo 'No log file'

In [ ]:
# Restart ngrok manually
import subprocess, os
os.chdir('/content')
print('Killing old ngrok...')
!pkill -9 ngrok
print('Starting fresh...')
!./ngrok tcp 22 > /tmp/ngrok_manual.log 2>&1 &
import time
time.sleep(5)
print('\nLog:')
!cat /tmp/ngrok_manual.log